File just to show how the web scraper works. It is used so that the user does not have to enter anything except a companys Ticker

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import yfinance as yf

In [15]:
def scrape_os_and_calulate_mkvalt(ticker):
    ticker = ticker.upper()
    try: 
        msft = yf.Ticker(ticker)
    except:
        print("The Ticker May be Delisted. Please check if the Ticker exists and make sure to enter correctly")
    scrape_url = 'https://finance.yahoo.com/quote'
    ticker_url = "{}/{}".format(scrape_url, ticker) +'/balance-sheet?p='+ ticker
    
    headers={'User-Agent': 'Custom'}

    response = requests.get(ticker_url,headers=headers )
    
    html = response.content
    soup = BeautifulSoup(html)
    
    ordinary_shares = soup.find(title = "Ordinary Shares Number")
    date = soup.find(class_='D(tbhg)').text
    if len(date) == 49:
        fyear_end_date = date[15:19] + "-" + date[9:11] + "-" + date[12:14]
    else:
        fyear_end_date = date[14:18] + "-" + date[9:10] + "-" + date[11:13]
    
    shares = []
    for i in ordinary_shares.parent.next_siblings:
        number = i.text.replace(",",'')
        shares.append(int(number))
    
    most_current_fyear_ordinary_shares = shares[0] / 100 
    
    j = msft.history(start = fyear_end_date)
    closing_price = j.Close.iloc[0]
    marketvalue = closing_price * most_current_fyear_ordinary_shares
    
    return [marketvalue, msft]

In [28]:
def clean_balance_sheet_for_feautures(msft, marketvalue):
    balanceSheet = msft.balance_sheet
    
    balanceSheet.drop(index=['Capital Surplus','Common Stock','Inventory','Other Stockholder Equity',
           'Property Plant Equipment','Good Will','Gains Losses Not Affecting Retained Earnings',
                              'Total Stockholder Equity','Retained Earnings',
                              'Long Term Investments','Net Tangible Assets','Accounts Payable',
                              'Other Current Liab','Other Assets','Total Assets','Other Liab'],inplace = True)
    first_col = balanceSheet.columns[0]
    current_numbers_needed = balanceSheet[first_col]
    current_numbers_needed = current_numbers_needed.to_dict()
    current_numbers_needed["Market Value"] = marketvalue
    return current_numbers_needed

In [29]:
marketvalue, msft = scrape_os_and_calulate_mkvalt('air')

In [30]:
clean_balance_sheet_for_feautures(msft,marketvalue)

{'Intangible Assets': 25500000.0,
 'Total Liab': 539400000.0,
 'Other Current Assets': 21600000.0,
 'Cash': 53500000.0,
 'Total Current Liabilities': 348200000.0,
 'Total Current Assets': 1007200000.0,
 'Net Receivables': 290300000.0,
 'Long Term Debt': 98900000.0,
 'Market Value': 17065.540632019045}